<a href="https://colab.research.google.com/github/deepanrajm/deep_learning/blob/master/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D



In [ ]:
df = tf.keras.datasets.fashion_mnist
# spliting the data into train and testing part
(train_df,train_labl),(test_df,test_labl) = df.load_data()

#scaling the train data
train_df=train_df/255.0
#scaling test data
test_df = test_df/255.0

#rehaping the images into equal dim.
train_df = train_df.reshape(len(train_df),28,28,1)
test_df = test_df.reshape(len(test_df),28,28,1)

In [ ]:
def build_model(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(28,28,1)),
    # adding second convolutional layer 
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        #activation function
        activation='relu'
    ),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    keras.layers.Dense(10, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model


#importing keras tuner
from kerastuner import RandomSearch
from kerastuner import Hyperband
from kerastuner import BayesianOptimization
#creating randomsearch object
tuner = RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 2)

#creating hyperband object
# tuner = Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=3,
#     factor=3,
#     hyperband_iterations=1
# )

#creating bayesian object
# tuner = BayesianOptimization(
#     build_model,
#     objective='val_accuracy',
#     max_trials=3,
#     num_initial_points=2
# )


# search best parameter
tuner.search(train_df,train_labl,epochs=3,validation_data=(train_df,train_labl))

In [ ]:
model=tuner.get_best_models(num_models=1)[0]
#summary of best model
print (model.summary())

model.fit(test_df,test_labl,
          epochs=10,
          validation_split=0.1,initial_epoch=3)